Note: This is just a sanity check to ensure that I can load up JSONs to study them, not a proper evaluation.

In [37]:
import os
import sys

sys.path.append('.')

logs_path = '../logs/'

In [44]:
# by default, pick the last experiment

experiment_number = 0
while os.path.exists(logs_path + f'exp_{experiment_number}/'):
    experiment_number += 1

experiment_path = logs_path + f'exp_{experiment_number - 1}/'

json_path = experiment_path + 'agent-logs.json'

print(f'Opening logs from Experiment {experiment_number - 1} at {json_path}')

Opening logs from Experiment 10 at ../logs/exp_10/agent-logs.json


In [55]:
with open(json_path, 'r') as f:
    # remove the last comma
    data = json.load(f)

In [41]:
json_data

'[{\n  "timestamp": "2025-01-20 21:31:30.541104",\n  "player": {\n    "name": "Player 1: orange",\n    "identity": "Crewmate"\n  },\n  "interaction": {\n    "type": "ACTION",\n    "prompt": {\n      "summarization": "no thought process has been made",\n      "all_info": "Game Time: 0/50\\nCurrent phase: Task phase\\nIn this phase, Crewmates should try to complete all tasks or identify Impostor. Impostor should try to kill Crewmates.\\n\\nCurrent Location: Cafeteria\\nPlayers in Cafeteria: Player 1: orange, Player 2: white, Player 3: blue, Player 4: yellow, Player 5: green\\n\\nObservation history:\\nNo observations have been made yet.\\n\\nAction history:\\nNo actions have been taken yet.\\n\\nYour Assigned Tasks:\\n1. common: Fix Wiring (Navigation) \\nPath: Cafeteria->Weapons->Navigation\\n2. short: Upload Data (Admin) \\nPath: Cafeteria->Admin\\n\\nAvailable actions:\\n1. MOVE from Cafeteria to Admin\\n2. MOVE from Cafeteria to Weapons\\n3. MOVE from Cafeteria to Upper Engine\\n4. M

In [27]:
logs

'{\n  "timestamp": "2025-01-20 17:37:49.610302",\n  "player": {\n    "name": "Player 1: red",\n    "identity": "Crewmate"\n  },\n  "interaction": {\n    "type": "ACTION",\n    "prompt": {\n      "summarization": "no thought process has been made",\n      "all_info": "Game Time: 0/50\\nCurrent phase: Task phase\\nIn this phase, Crewmates should try to complete all tasks or identify Impostor. Impostor should try to kill Crewmates.\\n\\nCurrent Location: Cafeteria\\nPlayers in Cafeteria: Player 1: red, Player 2: white, Player 3: lime, Player 4: purple, Player 5: green\\n\\nObservation history:\\nNo observations have been made yet.\\n\\nAction history:\\nNo actions have been taken yet.\\n\\nYour Assigned Tasks:\\n1. common: Fix Wiring (Cafeteria) \\nPath: You are already at the task location.\\n2. short: Download Data (Weapons) \\nPath: Cafeteria->Weapons\\n\\nAvailable actions:\\n1. MOVE from Cafeteria to Admin\\n2. MOVE from Cafeteria to Weapons\\n3. MOVE from Cafeteria to Upper Engine\\

In [28]:
from pandas import json_normalize
import json

In [33]:
df = json_normalize(
    data,
    # Main level columns
    meta=[
        'timestamp',
        ['player', 'name'],
        ['player', 'identity'],
        ['interaction', 'type'],
        ['interaction', 'response', 'Condensed Memory'],
        ['interaction', 'response', 'Thinking Process', 'thought'],
        ['interaction', 'response', 'Thinking Process', 'action']
    ]
)

In [34]:
df

,timestamp,player.name,player.identity,interaction.type,interaction.prompt.summarization,interaction.prompt.all_info,interaction.prompt.memory,interaction.response.Condensed Memory,interaction.response.Thinking Process,interaction.response.Action,interaction.response.Thinking Process.thought,interaction.response.Thinking Process.action
0,2025-01-20 19:16:03.930361,Player 1: cyan,Crewmate,ACTION,no thought process has been made,Game Time: 0/5\nCurrent phase: Task phase\nIn ...,no memory has been processed,No observations have been made yet.,I am currently in Cafeteria with Players 2 (br...,MOVE from Cafeteria to Weapons.,NaN,NaN
1,2025-01-20 19:16:08.865798,Player 2: brown,Impostor,ACTION,no thought process has been made,Game Time: 0/5\nCurrent phase: Task phase\nIn ...,no memory has been processed,I am currently in Cafeteria with Player 2: bro...,NaN,NaN,I have the opportunity to eliminate Player 3: ...,KILL Player 3: orange


In [35]:
# merge 'interaction.response.Action' with 'interaction.response.Thinking Process.action'
df['action'] = df['interaction.response.Thinking Process.action'].combine_first(df['interaction.response.Action'])
df = df.drop(columns=['interaction.response.Thinking Process.action', 'interaction.response.Action'])

# same, merge 'interaction.response.Thinking Process' with 'interaction.response.Thinking Process.thought' into 'thought'
df['thought'] = df['interaction.response.Thinking Process.thought'].combine_first(df['interaction.response.Thinking Process'])
df = df.drop(columns=['interaction.response.Thinking Process.thought', 'interaction.response.Thinking Process'])

In [36]:
df

,timestamp,player.name,player.identity,interaction.type,interaction.prompt.summarization,interaction.prompt.all_info,interaction.prompt.memory,interaction.response.Condensed Memory,action,thought
0,2025-01-20 19:16:03.930361,Player 1: cyan,Crewmate,ACTION,no thought process has been made,Game Time: 0/5\nCurrent phase: Task phase\nIn ...,no memory has been processed,No observations have been made yet.,MOVE from Cafeteria to Weapons.,I am currently in Cafeteria with Players 2 (br...
1,2025-01-20 19:16:08.865798,Player 2: brown,Impostor,ACTION,no thought process has been made,Game Time: 0/5\nCurrent phase: Task phase\nIn ...,no memory has been processed,I am currently in Cafeteria with Player 2: bro...,KILL Player 3: orange,I have the opportunity to eliminate Player 3: ...
